In [433]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import statsmodels.api as sm

#from skbio.stats.composition import clr
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.decomposition import PCA
from factor_analyzer import FactorAnalyzer

def color_big_red(val):
    color = 'red' if abs(val) >0.4 else 'black'
    return 'color: %s' % color

## Data 

In [434]:
data = pd.read_excel(r'C:\Users\boai-\Desktop\UnumAI\Colorado analysis\2020_CO_terms_cleaned.xlsx')

KeyboardInterrupt: 

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data[['Category', 'Term']].groupby('Category').count()

In [ ]:
# Setting up variables for each category.
# Not used but may be helpful
conspiracy = data[data['Category'].isin(['Conspiracy', 'Mueller/Russia'])]
corpreg_econ = data[data['Category'].isin(['Corporate Regulation','Economy'])]
crime = data[data['Category'].isin(['Criminal Justice', 'Public Safety', 'Guns'])]
Health= data[data['Category'].isin([ 'Healthcare'])]
education = data[data['Category'].isin(['Education'])]
electability = data[data['Category'].isin(['Electability'])]
environment = data[data['Category'].isin(['Environmental'])]
ethics = data[data['Category'].isin(['Ethics/Morals'])]
immigration = data[data['Category'].isin(['Immigration'])]
impeachment = data[data['Category'].isin(['Impeachment'])]
law_gov = data[data['Category'].isin(['Campaign Finance', 'Judiciary Branch', 'Generic Policy Queries', 'Voter Turnout', 'Election Security', 'Congress', 'Cybersecurity'])]
media = data[data['Category'].isin(['Media'])]
religion = data[data['Category'].isin(['Religion'])]
rephealth = data[data['Category'].isin(['Reproductive Health'])]
soceq = data[data['Category'].isin(['Social Equality'])]
taxes = data[data['Category'].isin(['Taxes/Spending'])]
trade_ag = data[data['Category'].isin(['Trade', 'Agriculture'])]
war = data[data['Category'].isin(['War'])]

In [ ]:
data.head()

# PCA
#### Run if PCA has not already been done

In [ ]:
df = data.drop(columns={"Term"}).fillna(0).groupby('Category').mean().T
df.drop(columns={'Space','Impeachment','State/Local'},inplace=True)


In [ ]:
df.head()

In [ ]:
df['tot'] = df.sum(axis=1)
for i in df.columns:
    df[i] = df[i] / df.tot
df.drop(columns={'tot'},inplace=True)

In [ ]:
fa = FactorAnalyzer(n_factors=14, rotation='varimax')
fa.fit(df)

factors = pd.DataFrame(fa.loadings_, index=list(df.columns)).style.applymap(color_big_red)
factors

In [ ]:
fa.get_factor_variance()

## Map data to PCA Categories and include dependents

In [ ]:
# PCA Category Mappings and Dependent variable files
pca_cats = pd.read_excel('2020 Dems Category Mapping.xlsx')
dependents = pd.read_excel('2020 Dependent.xlsx')

In [ ]:
dependents = dependents.rename(columns = {'Unnamed: 0':'Date'})

In [ ]:
dependents.head()

In [ ]:
reg_df = pd.merge(data,pca_cats,on='Category',how='left').drop(["Term"],axis = 1).groupby('Component').sum().T
reg_df = reg_df.drop('Unnamed: 0', axis = 0)
reg_df = reg_df.reset_index().rename(columns={'index':'Date'})
reg_df['Date'] = pd.to_datetime(reg_df['Date'], infer_datetime_format=True)
reg_df = pd.merge(reg_df,dependents,on='Date',how='inner')
reg_df = reg_df.fillna(0)

In [ ]:
# reg_df will be the clean dataframe used for modeling
reg_df.head()

In [ ]:
reg_df.columns

# Line plots for each variable
#### Identify spikes and unusual behavior

In [ ]:
#Original data, no roll
reg_df_date = reg_df.drop(['Date','Hickenlooper_raw','Hickenlooper_adj' ,'Gardner_raw','Gardner_adj','Electability'], axis = 1)
reg_cols = reg_df_date.columns
for i in reg_cols: 
    plt.plot(reg_df_date[i])
    plt.title(i)
    plt.show()

# Histograms
#### Distribution of each variable

In [ ]:
for i in reg_cols:
    plt.hist(reg_df_date[i])
    plt.title(i)
    plt.show()

# Rolling data
#### Use if you want to take a rolling average of the data

### rolling number give you a more steady average in term of weekly 

In [ ]:
roll_7 = reg_df.set_index(['Date'])
roll_7 = roll_7.rolling(7).mean()
roll_7 = roll_7.dropna()
roll_7 = roll_7.reset_index()
roll_7.head()

## Line plots for each variable after rolling data 
## Identify spikes and unusual behavior after rolling data 

In [ ]:
roll_plot = roll_7.drop(['Date','Hickenlooper_raw','Hickenlooper_adj' ,'Gardner_raw','Gardner_adj','Electability'], axis = 1)
roll_plot_cols = roll_plot.columns


In [ ]:
roll_plot_cols

# Rolling Train/Test and data exploration
#### Replace X variables dataframe with non rolled version if necessary

In [ ]:
# if not rolling data, rename this variable and replace throughout 
# x = reg_df.drop(...)
x_7 = roll_7.drop(columns={'Date','Hickenlooper_raw','Hickenlooper_adj' ,'Gardner_raw','Gardner_adj','Electability'})

Hickenlooper_adj = roll_7['Hickenlooper_adj']
Gardner_adj = roll_7['Gardner_adj']



x_train_Hickenlooper_adj, x_test_Hickenlooper_adj, y_train_Hickenlooper_adj, y_test_Hickenlooper_adj = train_test_split(x_7, Hickenlooper_adj, test_size=0.3, random_state=42)
x_train_Gardner_adj, x_test_Gardner_adj, y_train_Gardner_adj, y_test_Gardner_adj = train_test_split(x_7, Gardner_adj, test_size=0.3, random_state=42)

### Train Test random split

In [ ]:
#Hickenlooper_adj
reg_Hickenlooper_adj = LinearRegression().fit(x_train_Hickenlooper_adj, y_train_Hickenlooper_adj)
pred_train_Hickenlooper_adj = reg_Hickenlooper_adj.predict(x_train_Hickenlooper_adj)
pred_test_Hickenlooper_adj = reg_Hickenlooper_adj.predict(x_test_Hickenlooper_adj)
print('Hickenlooper_adj:: Train:', round(r2_score(y_train_Hickenlooper_adj,pred_train_Hickenlooper_adj),3),' Test:', round(r2_score(y_test_Hickenlooper_adj,pred_test_Hickenlooper_adj),3))

#Gardner_adj
reg_Gardner_adj = LinearRegression().fit(x_train_Gardner_adj, y_train_Gardner_adj)
pred_train_Gardner_adj = reg_Gardner_adj.predict(x_train_Gardner_adj)
pred_test_Gardner_adj = reg_Gardner_adj.predict(x_test_Gardner_adj)
print('Gardner_adj:: Train:', round(r2_score(y_train_Gardner_adj,pred_train_Gardner_adj),3),' Test:', round(r2_score(y_test_Gardner_adj,pred_test_Gardner_adj),3))



### Train Test chronological split

In [ ]:
Hickenlooper_adj_train = roll_7['Hickenlooper_adj'][:200]
Gardner_adj_train = roll_7['Gardner_adj'][:200]

Hickenlooper_adj_test = roll_7['Hickenlooper_adj'][200:]
Gardner_adj_test = roll_7['Gardner_adj'][200:]

x_7_train = x_7[:200]
x_7_test = x_7[200:]

In [ ]:
x_7_test.describe()

In [ ]:
#Hickenlooper_adj
reg_Hickenlooper_adj = LinearRegression().fit(x_7_train, Hickenlooper_adj_train)
pred_train_Hickenlooper_adj = reg_Hickenlooper_adj.predict(x_7_train)
pred_test_Hickenlooper_adj = reg_Hickenlooper_adj.predict(x_7_test)

print('Hickenlooper_adj:: Train:', round(r2_score(Hickenlooper_adj_train,pred_train_Hickenlooper_adj),3),' Test:', round(r2_score(Hickenlooper_adj_test,pred_test_Hickenlooper_adj),3))

#Gardner_adj
reg_Gardner_adj = LinearRegression().fit(x_7_train, Gardner_adj_train)
pred_train_Gardner_adj = reg_Gardner_adj.predict(x_7_train)
pred_test_Gardner_adj = reg_Gardner_adj.predict(x_7_test)
print('Gardner_adj:: Train:', round(r2_score(Gardner_adj_train,pred_train_Gardner_adj),3),' Test:', round(r2_score(Gardner_adj_test,pred_test_Gardner_adj),3))



In [ ]:
reg_Gardner_adj.score(x_7_test, Hickenlooper_adj_test)

## PCA after Rolling data 

In [ ]:
x_7.describe()

In [ ]:
x_7['tot'] = x_7.sum(axis=1)
for i in x_7.columns:
    x_7[i] = x_7[i] / x_7.tot
x_7.drop(columns={'tot'},inplace=True)

In [ ]:
fa_rd = FactorAnalyzer(n_factors=14, rotation='varimax')
fa_rd.fit(x_7)

factors = pd.DataFrame(fa_rd.loadings_, index=list(x_7.columns)).style.applymap(color_big_red)
factors

In [ ]:
fa_rd.get_factor_variance()

# Regression
#### Recommend naming variables per candidate as in steyer_baseline instead of just baseline
#### Adjust first line for model without a constant(intercept)

### Repeat per candidate -- Gardner 

In [ ]:
Gardner_adj_baseline = sm.add_constant(x_7)
mod = sm.OLS(Gardner_adj, Gardner_adj_baseline)
res = mod.fit()
print(res.summary())
# This is the baseline model with all variables

In [ ]:
Gardner_adj_baseline = Gardner_adj_baseline.drop(['Trade_Agr'], axis = 1)

In [ ]:
Gardner_adj_final_1 = Gardner_adj_baseline
mod = sm.OLS(Gardner_adj, Gardner_adj_final_1)
res = mod.fit()
res.summary()
#print(res.summary())
# This is the final model with significant variables remaining

In [ ]:
Gardner_adj_baseline = Gardner_adj_baseline.drop(['War'], axis = 1)

In [ ]:
Gardner_adj_final_2 = Gardner_adj_baseline
mod = sm.OLS(Gardner_adj, Gardner_adj_final_2)
res = mod.fit()
res.summary()

In [ ]:
Gardner_adj_baseline = Gardner_adj_baseline.drop(['Ethics'], axis = 1)

In [ ]:
Gardner_adj_final_3 = Gardner_adj_baseline
mod = sm.OLS(Gardner_adj, Gardner_adj_final_3)
res = mod.fit()
res.summary()

In [ ]:
Gardner_adj_baseline = Gardner_adj_baseline.drop(['ReprHealth'], axis = 1)

In [ ]:
Gardner_adj_final_4 = Gardner_adj_baseline
mod = sm.OLS(Gardner_adj, Gardner_adj_final_4)
res = mod.fit()
res.summary()

In [ ]:
Gardner_adj_baseline = Gardner_adj_baseline.drop(['Environment'], axis = 1)

In [ ]:
Gardner_adj_final_5 = Gardner_adj_baseline
mod = sm.OLS(Gardner_adj, Gardner_adj_final_5)
res = mod.fit()
res.summary()

In [ ]:
Gardner_adj_baseline = Gardner_adj_baseline.drop(['Religion'], axis = 1)

In [ ]:
Gardner_adj_final_6 = Gardner_adj_baseline
mod = sm.OLS(Gardner_adj, Gardner_adj_final_6)
res = mod.fit()
res.summary()

In [ ]:
# Correlation Matrix of significant variables
Gardner_adj_final_6.drop(['const'], axis = 1).corr()

In [ ]:
corrmat = Gardner_adj_final_6.drop(['const'], axis = 1).corr()
  
f, ax = plt.subplots(figsize =(10, 10)) 
sns.heatmap(corrmat, ax = ax, cmap ="coolwarm", linewidths = 0.1)

In [ ]:
# Variance Inflation Factor of significant variables
pd.Series([variance_inflation_factor(Gardner_adj_final_6.values, i) 
               for i in range(Gardner_adj_final_6.shape[1])], 
              index=Gardner_adj_final_6.columns).sort_values()

In [ ]:
Gardner_adj_final_6.describe()

## time split testing data and training data R2

In [ ]:
x_7.head()

In [ ]:
Gardner_adj_final_6.describe()

In [ ]:
x_train_Gardner_adj_final_6, x_test_Gardner_adj_final_6, y_train_Gardner_adj_final_6, y_test_Gardner_adj_final_6 = train_test_split(x_7, Gardner_adj_final_6, test_size=0.3, random_state=42)

In [ ]:
x_test_Gardner_adj_final_6.describe()

In [ ]:
#Gardner_adj_final_6
reg_Gardner_adj_final_6 = LinearRegression().fit(x_train_Gardner_adj_final_6, y_train_Gardner_adj_final_6)
post_train_Gardner_adj_final_6 = reg_Gardner_adj_final_6.predict(x_train_Gardner_adj_final_6)
post_test_Gardner_adj_final_6 = reg_Gardner_adj_final_6.predict(x_test_Gardner_adj_final_6)
print('Gardner_adj_final_6:: Train:', round(r2_score(y_train_Gardner_adj_final_6,post_train_Gardner_adj_final_6),3),' Test:', round(r2_score(y_test_Gardner_adj_final_6,post_test_Gardner_adj_final_6),3))



# ===================================

# ===================================

# Candidate Term Attribution Models
#### Models using only the terms that the candidates have used

In [ ]:
#Read in attribution file
attribution = pd.read_excel(r'C:\Users\boai-\Desktop\UnumAI\Colorado analysis\2020 CO Attribution.xlsx')
attribution.head()

In [ ]:
attribution['Term'] = attribution['Unnamed: 0']

In [ ]:
#join with original data
att_data = data.merge(attribution, on = 'Term', how = 'inner')
att_data = att_data.fillna(0)
att_data.shape
#att_data.head()

In [ ]:
att_data.head()

In [ ]:
Hickenlooper_att = att_data[att_data['John Hickenlooper'] > 0]
Hickenlooper_att = Hickenlooper_att.drop(['Unnamed: 0_x','Unnamed: 0_y','Cory Gardner','Other Dems'],axis = 1)

In [ ]:
Hickenlooper_att.head()

In [ ]:
H_group = Hickenlooper_att[['Category', 'Term']].groupby('Category').count()
H_group.head()

In [ ]:
H_group.shape

In [ ]:
Gardner_att = att_data[att_data['Cory Gardner'] > 0]
Gardner_att = Gardner_att.drop(['Unnamed: 0_x','Unnamed: 0_y','John Hickenlooper','Other Dems'],axis = 1)

In [ ]:
Gardner_att.head()

In [ ]:
G_group = Gardner_att[['Category', 'Term']].groupby('Category').count()
G_group.head()

In [ ]:
G_group.shape

In [ ]:
result = pd.merge(H_group, G_group, how='left', on=['Category'])
result.columns = ['John Hicklooper','Cory Gardner']
result.head()

In [ ]:
#result = result.reset_index()
#result = result.rename(columns = { 'index': "Category"})
#result.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


#df3=result[['Cory Gardner','John Hicklooper']].set_index('Category')
#print(df3)
result.plot.bar()             # standard bar chart
plt.tight_layout()
plt.show()


In [ ]:
reg_df_Gardner = pd.merge(Gardner_att,pca_cats,on='Category',how='left').drop(["Term"],axis = 1).groupby('Component').sum().T


In [ ]:
reg_df_Gardner.head()

In [ ]:
#reg_df_Gardner = reg_df_Gardner.drop('Unnamed: 0', axis = 0)
reg_df_Gardner = reg_df_Gardner.reset_index().rename(columns={'index':'Date'})
reg_df_Gardner['Date'] = pd.to_datetime(reg_df_Gardner['Date'], infer_datetime_format=True)
reg_df_Gardner = pd.merge(reg_df_Gardner,dependents,on='Date',how='inner')
reg_df_Gardner = reg_df_Gardner.fillna(0)
reg_df_Gardner.head()

In [ ]:
reg_df_Gardner = reg_df_Gardner.drop(['Date','Hickenlooper_raw','Hickenlooper_adj' ,'Gardner_raw','Gardner_adj','Electability'], axis = 1)


In [ ]:
reg_df_Gardner.head()

# =======================================

# Gardner - attribution model

In [ ]:
# Removing Electability, Date, and other Candidates
Gardner_att_data = reg_df_Gardner

In [ ]:
Gardner_att_baseline.shape

In [ ]:
Gardner_att.shape

In [ ]:
Gardner_adj.shape

In [ ]:
Gardner_att_baseline = sm.add_constant(Gardner_att_data)
mod = sm.OLS(Gardner_adj, Gardner_att_baseline)
res = mod.fit()
print(res.summary())

In [ ]:
steyer_att_final = sm.add_constant(steyer_att_data.drop([***Drop Columns Here***], axis = 1))
mod = sm.OLS(steyer, steyer_att_final)
res = mod.fit()
res.summary()

In [ ]:
steyer_att_final.corr()

In [ ]:
pd.Series([variance_inflation_factor(steyer_att_final.values, i) 
               for i in range(steyer_att_final.shape[1])], 
              index=steyer_att_final.columns).sort_values()

## regression model for term Immpeachment 

In [463]:
pca_cat = pd.read_excel('2020 Dems Category Mapping.xlsx')
dependents = pd.read_excel('2020 Dependent.xlsx')

In [464]:
data = pd.read_excel(r'C:\Users\boai-\Desktop\UnumAI\Colorado analysis\2020_CO_terms_Law_Govt_cleaned.xlsx')

In [465]:
dependents = dependents.rename(columns = {'Unnamed: 0':'Date'})

In [466]:
dependents.shape

(213, 5)

In [467]:
dependents

,Date,Hickenlooper_raw,Gardner_raw,Hickenlooper_adj,Gardner_adj
0,2019-02-15,0.431907,0.568093,0.474177,0.525823
1,2019-02-16,0.431490,0.568510,0.474032,0.525968
2,2019-02-17,0.430931,0.569069,0.473836,0.526164
3,2019-02-18,0.430737,0.569263,0.473768,0.526232
4,2019-02-19,0.432308,0.567692,0.474318,0.525682
...,...,...,...,...,...
208,2019-09-11,0.561576,0.438424,0.519562,0.480438
209,2019-09-12,0.561216,0.438784,0.519436,0.480564
210,2019-09-13,0.561136,0.438864,0.519408,0.480592
211,2019-09-14,0.561026,0.438974,0.519369,0.480631


In [468]:
data.head()

,Unnamed: 0,Term,Category,2019-02-15 00:00:00,2019-02-16 00:00:00,2019-02-17 00:00:00,2019-02-18 00:00:00,2019-02-19 00:00:00,2019-02-20 00:00:00,2019-02-21 00:00:00,...,2019-10-06 00:00:00,2019-10-07 00:00:00,2019-10-08 00:00:00,2019-10-09 00:00:00,2019-10-10 00:00:00,2019-10-11 00:00:00,2019-10-12 00:00:00,2019-10-13 00:00:00,2019-10-14 00:00:00,2019-10-15 00:00:00
0,0,115th congress,Law_Govt,0.000008,0.000031,0.000011,0.000010,0.000023,0.000009,NaN,...,NaN,0.000009,NaN,0.000008,NaN,0.000009,NaN,NaN,NaN,0.000016
1,1,116th congress,Law_Govt,0.000035,NaN,NaN,0.000027,0.000014,0.000024,0.000023,...,0.000046,0.000034,0.000024,0.000008,0.000016,0.000009,NaN,0.000013,0.000027,0.000022
2,2,2 house,Law_Govt,0.000042,0.000013,0.000018,0.000045,0.000046,0.000039,0.000029,...,0.000036,0.000079,0.000071,0.000062,0.000055,0.000058,0.000058,0.000019,0.000063,0.000052
3,3,3 branches of government,Law_Govt,0.000062,0.000031,0.000055,0.000062,0.000062,0.000052,0.000055,...,0.000065,0.000077,0.000054,0.000070,0.000086,0.000061,0.000024,0.000047,0.000044,0.000076
4,4,achieved,Law_Govt,0.000011,0.000015,0.000020,0.000011,NaN,0.000024,0.000011,...,NaN,0.000014,0.000011,0.000024,NaN,0.000013,0.000013,NaN,0.000012,NaN


In [469]:
# importing pandas as pd 
import pandas as pd 
data = data.drop('Unnamed: 0',axis =1)
data = data.drop('Category',axis =1)


In [470]:
data = data.transpose()
new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header
data = data.reset_index()
data = data.rename(columns = { 'index': "Date"})
data.head()

Term,Date,115th congress,116th congress,2 house,3 branches of government,achieved,achievements,advocate,advocates,advocating,...,sponsors,stacey abrams,subsidiary,us companies,use act,vote,vote of confidence,vote today,voted,voter
0,2019-02-15,7.59807e-06,3.49511e-05,4.24407e-05,6.24127e-05,1.09222e-05,2.73056e-05,0.000149791,4.99302e-05,1.24825e-05,...,5.46112e-06,1.31067e-05,7.82488e-06,1.04021e-05,1.17373e-05,0.000436889,NaN,3.12995e-05,0.000227182,2.91259e-05
1,2019-02-16,3.07001e-05,NaN,1.28864e-05,3.07001e-05,1.52603e-05,NaN,0.000142337,4.74455e-05,1.58152e-05,...,1.14452e-05,2.71823e-05,1.44973e-05,9.31966e-06,1.28864e-05,0.000260951,NaN,6.44322e-06,8.11846e-05,3.47934e-05
2,2019-02-17,1.12353e-05,NaN,1.75856e-05,5.51549e-05,2.02235e-05,4.44916e-05,0.000242682,4.04469e-05,2.02235e-05,...,1.96617e-05,1.44453e-05,1.41564e-05,NaN,1.41564e-05,0.000219088,8.08939e-06,NaN,5.05587e-05,5.05587e-05
3,2019-02-18,9.66475e-06,2.73834e-05,4.54449e-05,6.16128e-05,1.06e-05,1.06e-05,0.000213591,6.57203e-05,3.28601e-05,...,1.23226e-05,5.86788e-06,1.17358e-05,1.91684e-05,2.05376e-05,0.000205376,5.86788e-06,5.86788e-06,1.82556e-05,2.28195e-05
4,2019-02-19,2.29852e-05,1.42289e-05,4.56011e-05,6.18222e-05,NaN,3.3632e-05,0.000211655,3.73509e-05,1.24503e-05,...,1.06717e-05,4.1501e-06,5.10782e-06,9.05476e-06,1.66004e-05,0.000209165,NaN,5.10782e-06,6.8149e-05,5.24223e-05


In [471]:
#data[212:]
data = data.drop(data.index[213:])


In [472]:
all_df_list = [data, dependents]


In [473]:
data.shape

(213, 694)

In [474]:
dependents.shape

(213, 5)

In [488]:
reg_df = pd.merge(data,dependents,on='Date',how='left')

In [489]:
reg_df = reg_df.fillna(0)

In [490]:
reg_df.head()

,Date,115th congress,116th congress,2 house,3 branches of government,achieved,achievements,advocate,advocates,advocating,...,use act,vote,vote of confidence,vote today,voted,voter,Hickenlooper_raw,Gardner_raw,Hickenlooper_adj,Gardner_adj
0,2019-02-15,0.000008,0.000035,0.000042,0.000062,0.000011,0.000027,0.000150,0.000050,0.000012,...,0.000012,0.000437,0.000000,0.000031,0.000227,0.000029,0.431907,0.568093,0.474177,0.525823
1,2019-02-16,0.000031,0.000000,0.000013,0.000031,0.000015,0.000000,0.000142,0.000047,0.000016,...,0.000013,0.000261,0.000000,0.000006,0.000081,0.000035,0.431490,0.568510,0.474032,0.525968
2,2019-02-17,0.000011,0.000000,0.000018,0.000055,0.000020,0.000044,0.000243,0.000040,0.000020,...,0.000014,0.000219,0.000008,0.000000,0.000051,0.000051,0.430931,0.569069,0.473836,0.526164
3,2019-02-18,0.000010,0.000027,0.000045,0.000062,0.000011,0.000011,0.000214,0.000066,0.000033,...,0.000021,0.000205,0.000006,0.000006,0.000018,0.000023,0.430737,0.569263,0.473768,0.526232
4,2019-02-19,0.000023,0.000014,0.000046,0.000062,0.000000,0.000034,0.000212,0.000037,0.000012,...,0.000017,0.000209,0.000000,0.000005,0.000068,0.000052,0.432308,0.567692,0.474318,0.525682


In [491]:
# if not rolling data, rename this variable and replace throughout 
# x = reg_df.drop(...)
x_7 = reg_df.drop(columns={'Date','Hickenlooper_raw','Hickenlooper_adj' ,'Gardner_raw','Gardner_adj'}, axis = 1)

Hickenlooper_adj = reg_df['Hickenlooper_adj']
Gardner_adj = reg_df['Gardner_adj']



x_train_Hickenlooper_adj, x_test_Hickenlooper_adj, y_train_Hickenlooper_adj, y_test_Hickenlooper_adj = train_test_split(x_7, Hickenlooper_adj, test_size=0.3, random_state=42)
x_train_Gardner_adj, x_test_Gardner_adj, y_train_Gardner_adj, y_test_Gardner_adj = train_test_split(x_7, Gardner_adj, test_size=0.3, random_state=42)

In [492]:
#Hickenlooper_adj
reg_Hickenlooper_adj = LinearRegression().fit(x_train_Hickenlooper_adj, y_train_Hickenlooper_adj)
pred_train_Hickenlooper_adj = reg_Hickenlooper_adj.predict(x_train_Hickenlooper_adj)
pred_test_Hickenlooper_adj = reg_Hickenlooper_adj.predict(x_test_Hickenlooper_adj)
print('Hickenlooper_adj:: Train:', round(r2_score(y_train_Hickenlooper_adj,pred_train_Hickenlooper_adj),3),' Test:', round(r2_score(y_test_Hickenlooper_adj,pred_test_Hickenlooper_adj),3))

#Gardner_adj
reg_Gardner_adj = LinearRegression().fit(x_train_Gardner_adj, y_train_Gardner_adj)
pred_train_Gardner_adj = reg_Gardner_adj.predict(x_train_Gardner_adj)
pred_test_Gardner_adj = reg_Gardner_adj.predict(x_test_Gardner_adj)
print('Gardner_adj:: Train:', round(r2_score(y_train_Gardner_adj,pred_train_Gardner_adj),3),' Test:', round(r2_score(y_test_Gardner_adj,pred_test_Gardner_adj),3))



Hickenlooper_adj:: Train: 1.0  Test: 0.364
Gardner_adj:: Train: 1.0  Test: 0.364


In [495]:
Hickenlooper_adj_train = roll_7['Hickenlooper_adj'][:200]
Gardner_adj_train = roll_7['Gardner_adj'][:200]

Hickenlooper_adj_test = roll_7['Hickenlooper_adj'][200:]
Gardner_adj_test = roll_7['Gardner_adj'][200:]

x_7_train = x_7[:200]
x_7_test = x_7[200:]

In [496]:
x_7_test.describe()

,115th congress,116th congress,2 house,3 branches of government,achieved,achievements,advocate,advocates,advocating,affairs,...,sponsors,stacey abrams,subsidiary,us companies,use act,vote,vote of confidence,vote today,voted,voter
count,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,...,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
mean,0.000002,0.000016,0.000045,0.000103,0.000010,0.000018,0.000195,0.000046,0.000017,0.000600,...,0.000012,0.000007,0.000006,0.000008,0.000016,0.000227,0.000006,0.000008,0.000031,0.000029
std,0.000004,0.000010,0.000015,0.000060,0.000007,0.000016,0.000046,0.000015,0.000016,0.000090,...,0.000003,0.000007,0.000004,0.000005,0.000005,0.000070,0.000003,0.000005,0.000015,0.000014
min,0.000000,0.000000,0.000019,0.000037,0.000000,0.000000,0.000140,0.000023,0.000000,0.000449,...,0.000006,0.000000,0.000000,0.000000,0.000007,0.000123,0.000000,0.000000,0.000005,0.000014
25%,0.000000,0.000015,0.000036,0.000083,0.000000,0.000011,0.000148,0.000034,0.000014,0.000569,...,0.000009,0.000000,0.000006,0.000005,0.000013,0.000182,0.000006,0.000006,0.000021,0.000019
50%,0.000000,0.000017,0.000046,0.000092,0.000011,0.000016,0.000182,0.000044,0.000016,0.000628,...,0.000012,0.000006,0.000006,0.000009,0.000015,0.000211,0.000006,0.000006,0.000028,0.000026
75%,0.000000,0.000018,0.000056,0.000124,0.000014,0.000019,0.000240,0.000056,0.000017,0.000643,...,0.000015,0.000007,0.000009,0.000011,0.000019,0.000275,0.000009,0.000010,0.000042,0.000037
max,0.000010,0.000039,0.000069,0.000268,0.000021,0.000060,0.000250,0.000066,0.000054,0.000747,...,0.000018,0.000019,0.000014,0.000018,0.000028,0.000385,0.000011,0.000016,0.000059,0.000062


In [498]:
Gardner_adj_baseline = sm.add_constant(x_7)
mod = sm.OLS(Gardner_adj, Gardner_adj_baseline)
res = mod.fit()
print(res.summary())
# This is the baseline model with all variables

C:\Users\boai-\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning:

Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.



                            OLS Regression Results                            
Dep. Variable:            Gardner_adj   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                     0.000
Date:                Mon, 18 Nov 2019   Prob (F-statistic):                nan
Time:                        15:28:20   Log-Likelihood:                 7047.1
No. Observations:                 213   AIC:                        -1.367e+04
Df Residuals:                       0   BIC:                        -1.295e+04
Df Model:                         212                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

C:\Users\boai-\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1648: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\boai-\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1649: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\boai-\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1665: RuntimeWarning:

divide by zero encountered in double_scalars

C:\Users\boai-\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1578: RuntimeWarning:

divide by zero encountered in double_scalars

C:\Users\boai-\Anaconda3\lib\site-packages\statsmodels\base\model.py:1376: RuntimeWarning:

invalid value encountered in multiply

C:\Users\boai-\AppData\Roaming\Python\Python37\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning:

invalid value encountered in greater

C:\Users\boai-\AppData\Roaming\Python\Python37\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeW